## Cleaning And Aggregating All the Data
This notebooked will be the template for the cleaning and aggregation of all the data for the website. This will make sure all data types are fixed. And everything is accounted for. I think this is important before we actually start to produve the dashboard

In [16]:
import pandas as pd
import numpy as np
import math
import os
import json
import openpyxl
import datetime
import re
from sklearn.preprocessing import MinMaxScaler

## School Location Coordinates 
The first thing we have to do is only show information about the active schools and the active students for all the data sets. This is important to show case. We will however uses the indexes developed by the dropouts to see if there are differences. This is will be Brimar's job.

### Missing Data 
Cluster: Most of the values are gone this is because they only exist for the csr schools.
Pathan Colony has everything missing.
Village name has 16 that are missing because these exist only for the sef school.
So missing data is all accounted for. 

### Fixing Coordinates
Some of the coordinates have bad values. We will not care about the actual cooridinate values but instead will fix them to decimal form and as TTWF gets better data we will fix it. 

These coordinates are written in decimal degrees: #degrees°#minutes'#".
We can convert them to decimals with this formula. 

$$ 
    \text{Decimal Degrees} = \text{Degrees} + \frac{\text{Minutes}}{60} + \frac{\text{Seconds}}{3600}
$$

In [17]:
# first we import the school locations 

school_locations_sef = pd.read_excel(os.path.join(os.getcwd(), '..', 'cleaned_TTWF_DATA', 'School_Locations.xlsx'), sheet_name='SEF Schools',  header=1)
school_locations_csr = pd.read_excel(os.path.join(os.getcwd(), '..', 'cleaned_TTWF_DATA', 'School_Locations.xlsx'), sheet_name='CSR Schools', header = 1)

# the first thing we should do is concatenate both of the data frames
school_locations_sef['School_Type'] = 'SEF'
school_locations_csr['School_Type'] = 'CSR'
school_locations = pd.concat([school_locations_csr, school_locations_sef])

# rename the columns
school_locations.rename(columns={'GPS':'Lat', 'Unnamed: 5':'Long'}, inplace = True)

# get rid of the Sr.No
school_locations.drop(columns = ['S.No'], inplace = True)

# clean coordinates 
def clean_coords(coords):
    # first check if it is valid
    if isinstance(coords, int):
        string_coords = str(coords)
        return float(string_coords[:2] + '.' + string_coords[3:])
    if isinstance(coords, float):
        return coords
    # then try the edge case 
    elif str(coords).strip()[-1] == ',':
        return float(coords.strip()[:-1])
    # then check if the last one is a comma 
    elif coords[-1] == ",":
        return float(coords[:-1])
    # then if there is a column and space 
    elif coords[-2] == ", ":
        return float(coords[:-2])
    # then we check if the value has two spaces 
    elif coords.count(" ") == 2:
        first_space = coords.find(" ")
        second_space = coords.find(" ", first_space + 1)
        degree = coords[:first_space]
        minutes = coords[first_space + 1:second_space]
        seconds = coords[second_space + 1:]
        final_coordinate = int(degree) + (int(minutes) / 60) + (float(seconds) / 3600)
        return final_coordinate
    # then we fix it for values with two decimal places
    elif coords.count(".") == 2: 
        first_dot = coords.find('.')     
        second_dot = coords.find('.', first_dot + 1)  
        degree = coords[:first_dot]
        minutes = coords[first_dot + 1:second_dot]
        seconds = coords[second_dot + 1:]
        final_coordinate = int(degree) + (int(minutes) / 60) + (int(seconds) / 3600)
        return final_coordinate
    # if there is a space and a .
    elif (" " in coords) and ("." in coords):
        first_space = coords.find(" ")
        first_dot = coords.find(".", first_space + 1)
        degree = coords[:first_space]
        minutes = coords[first_space + 1:first_dot]
        seconds = coords[first_dot + 1:]
        final_coordinate = int(degree) + (int(minutes) / 60) + (float(seconds) / 3600)
        return final_coordinate
    # then we fix this for the degree
    elif ('°' in coords):
        first_dot = coords.find("°")
        first_app = coords.find("'")
        degree = coords[:first_dot]
        minutes = coords[first_dot + 1:first_app]
        seconds = coords[first_app + 1:]
        final_coordinate = int(degree) + (int(minutes) / 60) + (float(seconds) / 3600)
        return final_coordinate 
    else:
        print("THERE WAS AN ERROR FOR", coords)
        return coords
                
school_locations['Lat_cleaned'] = school_locations['Lat'].apply(clean_coords)
school_locations['Long_cleaned'] = school_locations['Long'].apply(clean_coords)

# then drop the old coordinates 
school_locations.drop(columns = ["Lat", 'Long'], inplace = True)

## Master School List
This is the data that gives the most information to the schools. We will compare the coordinates from above to make sure they are alligned and also try to use them to fill in some of the ones that we have. The way we fill in the coordinates is by looking at the corrasponding division or district and then take the mean and standard deviation of each of the groups. Then we take a random value based off of this and multiply by a small constant to ensure that the values are cleaned. This makes sure that all of the schools are actually accounted for. 

### Things to add to the School Card 
* The Launch Date: Fix This with unknown. 
* The Type: 
* Connectivity: Fill in all with status not checked 
* CCTV: Fill in all the Nones with No

### Missing Status 
The next thing we have to do is see how we can fill in the missing status. I think the best way is to pull from the ENUMA log data set. If any users in the activty log have a student that is active in registration then we will assume that the school is active. We will be using the school code to match the schools but this may be false. It seems like the schoolArea and the schoolCluster in the ENUMA data map to the master_school_list Division and District respectively. So we can try to infer these values in order to connect the ENUMA data to the TTWF. Instead what we will do is create a bin called miscellaneous. Then we will assign random coordinates to each of the schools in this miscellaneous to then be able to assign points to everything

Then we have a bridge between the schools and the ENUMA data! This will be a huge part of the actual calculations for later. I am going to assign all the missing divisions in the ENUMA data to Sehwan just cause it is the only point on the map right now with no ENUMA users. Then the next thing we would have to do is figure out all the school coordinates for the values we just computed, we should also show in the box if these coordinates were manually filled in and that they should be addressed later.The coordinates will just be imputed by taking the mean of all the Sehwan coordinates then we will randomly jitter like we did above.

In [18]:
# read in the data 
master_school_list = pd.read_csv(os.path.join(os.getcwd(), '..', 'cleaned_TTWF_DATA', 'cleaned_master_school_list.csv'))

# replace missing with the none type
master_school_list.replace({'missing':None}, inplace = True)

# replace the one coordinate that got read wierd 
master_school_list.loc[(~master_school_list['Loc_X'].isna()) & (master_school_list['Loc_X_clean'].isna()), ['Loc_X_clean']] = clean_coords(
    master_school_list.loc[(~master_school_list['Loc_X'].isna()) & (master_school_list['Loc_X_clean'].isna()), :]['Loc_X'].values[0]
)


def clean_ordinal_dates(date_str):
    # Remove ordinal suffixes like "23rd", "19th", etc.
    if date_str == None:
        return date_str
    fixed_string = re.sub(r'(\d{1,2})(st|nd|rd|th)', r'\1', date_str)

    # then we strip all white space before and after 
    fixed_string = fixed_string.strip()
    # try to change to datetime otherwise print it out
    try:
        if fixed_string == '30 Novermber 2024':
            try:
                return pd.to_datetime('30 November 2024')
            except: 
                print(fixed_string, 'but passed')
        return pd.to_datetime(fixed_string, dayfirst=True) 
    except:
        print(fixed_string)

master_school_list['fixed_launch_date'] = master_school_list['Launch_Date'].apply(clean_ordinal_dates)

# then we calculcate the age of the school in months 
# replace the rest with missing
def calculate_school_age(launch_date):
    if pd.isnull(launch_date):
        return "missing"
    else:
        today = pd.to_datetime("today")
        age_years = (today - launch_date).days // 31
        return age_years

master_school_list['school_age_months'] = master_school_list['fixed_launch_date'].apply(calculate_school_age)

# then next thing we have to ensure is to impute the value of the missing coordinates
# we will do this soley based on the division
# we will also do this based on the district as well
# So we will impute the one missing district from the school locations
master_school_list.loc[master_school_list['District'].isna(), ['District']] = 'Malir'

# we first create new columns to indicate when we simulate the coordinates 
master_school_list['Missing_X'] = master_school_list['Loc_X_clean'].apply(lambda x: 1 if pd.isnull(x) else 0)

master_school_list['Missing_Y'] = master_school_list['Loc_Y_clean'].apply(lambda x: 1 if pd.isnull(x) else 0)

# convert the locations into float 
master_school_list['Loc_X_clean'] = master_school_list['Loc_X_clean'].astype(float)
master_school_list['Loc_Y_clean'] = master_school_list['Loc_Y_clean'].astype(float)

# Impute function using group stats
def random_impute_from_group(row, group_df, col, global_df):
    group_mean = group_df[col].mean()
    group_std = group_df[col].std()

    # If group has no valid data 
    if pd.isnull(group_mean):
        group_mean = global_df[col].mean()
        group_std = global_df[col].std()

    # If std is missing or zero, set to a small default
    if pd.isnull(group_std) or group_std == 0:
        group_std = 0.05

    return np.random.normal(loc=group_mean, scale=group_std * 0.1)

# Create division-level imputed columns
def impute_by_division(df, col):
    result = []
    for _, row in df.iterrows():
        if pd.notnull(row[col]):
            result.append(row[col])
        else:
            group = df[df['Division'] == row['Division']]
            result.append(random_impute_from_group(row, group, col, df))
    return result

def impute_by_district(df, col):
    result = []
    for _, row in df.iterrows():
        if pd.notnull(row[col]):
            result.append(row[col])
        else:
            group = df[df['District'] == row['District']]
            result.append(random_impute_from_group(row, group, col, df))
    return result


# 5. Generate new columns
master_school_list['Loc_X_division_imputed'] = impute_by_division(master_school_list, 'Loc_X_clean')
master_school_list['Loc_Y_division_imputed'] = impute_by_division(master_school_list, 'Loc_Y_clean')
master_school_list['Loc_X_district_imputed'] = impute_by_district(master_school_list, 'Loc_X_clean')
master_school_list['Loc_Y_district_imputed'] = impute_by_district(master_school_list, 'Loc_Y_clean')

# fill in the missing values for the for the CCTV 
# assuming that all missing means that they do not have CCTV. We will need to check this later
master_school_list['CCTV'] = master_school_list['CCTV'].apply(lambda x: 'No' if pd.isnull(x) else 'Yes')
# do the same for the connectivity
master_school_list['Connectivity'] = master_school_list['Connectivity'].apply(lambda x: 'Not Recorded' if x == None else x)

# create an indicator is the status is missing 
master_school_list['Missing_Status'] = master_school_list['Status'].apply(lambda x:1 if x == None else 0)

# we create the school code 3 for the master learning data 
master_school_list['School_Code_3'] = master_school_list['School_Id'].apply(lambda x:x[:3]).astype(int)

## ENUMA Users Data
We will try imputing the status of the schools based off of the ENUMA Users data. We will check if there is a single student that has active registration in the ENUMA data, otherwise we will fill in the Status as inactive. 

In [19]:
# merge the full users with the master_school_list and only keep the division column
enuma_users = pd.read_csv(os.path.join(os.getcwd(), '..', 'ENUMA_DATA', 'enuma_users_df.csv'))
 
columns_to_keep = enuma_users.columns.tolist()
columns_to_keep.append('Division')
columns_to_keep.append('District')
columns_to_keep.append('Loc_X_division_imputed')
columns_to_keep.append('Loc_Y_division_imputed')
enuma_users = enuma_users.merge(master_school_list, left_on = 'schoolUID', right_on='School_Code_3', how = 'left').loc[:, columns_to_keep]

# now we understand why there are missing values, it looks like we are going to introduce these values to the information 
# we will first put an indicator where we have missing values to undertand where things have been imputed
# we are just going to map everything for the divisions that are missing to the divisions with the least amount of schools 
# first create a marker for the ones that we imput 
enuma_users['missing_division'] = enuma_users['Division'].apply(lambda x:1 if pd.isna(x) == True else 0)

# impute all missing schools in here with sehwan
enuma_users['Division'] = enuma_users['Division'].fillna('Sehwan')

In [20]:
# Load the district coordinates JSON (list of dicts)
with open(os.path.join(os.getcwd(), '..', 'assets', "district_coords.json")) as f:
    district_data_list = json.load(f)

# Convert to dict with float lat/lon values
district_data = {
    entry["name"]: {
        "lat": float(entry["lat"]),
        "lon": float(entry["long"])
    }
    for entry in district_data_list
}

print(district_data)

# Jittering function: preserves real coords if present, else jitter around district center
def jitter_from_district_coords(row, jitter_std=0.05):
    # If real coordinates already exist, use them
    if pd.notnull(row.get("Loc_Y_division_imputed")) and pd.notnull(row.get("Loc_X_division_imputed")):
        return pd.Series([row["Loc_X_division_imputed"], row["Loc_Y_division_imputed"]])
    
    coords = district_data.get(row["Division"])
    if coords is None:
        return pd.Series([np.nan, np.nan])
    
    lat = coords["lat"]
    lon = coords["lon"]
    
    jittered_lat = np.random.normal(loc=lat, scale=jitter_std)
    jittered_lon = np.random.normal(loc=lon, scale=jitter_std)
    
    return pd.Series([jittered_lat, jittered_lon])

# Apply to ENUMA users
enuma_users[['final_jittered_Loc_X', 'final_jittered_Loc_Y']] = enuma_users.apply(
    jitter_from_district_coords, axis=1
)

# OPTIONAL: clip coordinates to Pakistan's bounds for safety
# enuma_users['final_jittered_Loc_X'] = enuma_users['final_jittered_Loc_X'].clip(60.0, 77.5)
#enuma_users['final_jittered_Loc_Y'] = enuma_users['final_jittered_Loc_Y'].clip(23.5, 37.5)





{'Banbhore': {'lat': 24.45, 'lon': 67.31}, 'Karachi': {'lat': 24.8607, 'lon': 67.0011}, 'Hyderabad': {'lat': 25.396, 'lon': 68.37}, 'Sukkur': {'lat': 27.7172, 'lon': 68.8574}, 'Larkana': {'lat': 27.56, 'lon': 67.25}, 'Sehwan': {'lat': 26.248, 'lon': 68.4072}, 'Mirpurkhas': {'lat': 25.525, 'lon': 70.5}}


## Learner Profile
We will be using this for the main home page only. There is not sufficient data to map this with the ENUMA data, thus it is difficult to use. We will only be using ENUMA for the kids actual metrics. I think this is the biggest problem we need to solve right now with the data in general. Once this is done then the dashboard will have a lot more validity. 

## inactive_schools_df_activity_log
This data frame will also only be used in the main kpi, this is soley based on the ENUMA data. We will work around this for later. 

## schools_not_in_activity_log
This is the schools not in the activity log here as well, we will be using this too for only the home page. 


## combined_missing_schools_curr
This tells which of the schools in the user log that are active that are not active in the curriculum log. This should be studied later as well maybe we can deem them as inactive. 

One thing is settling the differences between the ENUMA and the TTWF data.

## Next Big Goal 
Now the rest of the data will be filled in with the ENUMA data. We will attempt to look combine all the scores from all the students to actually compute the values of the dots on all of our maps. If we accomplish this today then we are pretty much able to build the first protype of the entire webapp. At this point we only want to look at the active students. 

## Activity Log Index 
This data set will be used to track the progress of each of the kids. We need some way to extract an index from this overall data to create an engagement index from this. Then we will be able to combine this with the other two data frames. 

We will aggregate all of this data by doing the following. We will create three indexes, one will be the engagement index which composes off of things like the num lessons attempted and the num lessons completed. The other will be the active days the students is preseset, and we can also include the average session gaps. Then we will create a performance index based on the categories that we have made and check the progress as will. This will all be normalized by the active days. We ran a statistial test on the two categories to determine whether the actual values will be good. We determined whether the tools we good by running a one way anova test with the values we have for the discretized categories of whether the kids are good or not.


In [21]:
# we first need to read in the data from the activity log this is the index data
activity_log_index = pd.read_csv(os.path.join(os.getcwd(), '..', 'ENUMA_DATA', 'full_student_index_activity_log.csv'))

# we will first normalize the features by the active days 
activity_log_index.loc[:, 'lessons_attempted_per_day'] = activity_log_index.loc[:, 'num_lessons_attempted'] / activity_log_index.loc[:,'active_days']
activity_log_index.loc[:, 'lessons_completed_per_day'] = activity_log_index.loc[:, 'num_lessons_completed'] / activity_log_index.loc[:, 'active_days']

# we are then going to impute values for the missing data with the median 
gap_cols = ['avg_session_gap', 'max_session_gap', 'last_session_gap', 'num_gaps_gt_3', 'num_gaps_gt_7']
for col in gap_cols:
    activity_log_index.loc[:, col] = activity_log_index[col].fillna(activity_log_index[col].median())

# we then normalize all of the values as well 
scaler = MinMaxScaler()
activity_log_index.loc[:, ['lessons_completed_per_day', 'completion_rate', 'avg_session_gap']] = scaler.fit_transform(
    activity_log_index[['lessons_completed_per_day', 'completion_rate', 'avg_session_gap']]
)

# we then create a consistensy score 
activity_log_index.loc[:, 'consistency_score'] = 1 - activity_log_index['avg_session_gap']

activity_log_index.loc[:, 'learning_index'] = (
    0.4 * activity_log_index['lessons_completed_per_day'] +
    0.8 * activity_log_index['completion_rate'] +
    0.2 * activity_log_index['consistency_score']
)

# rename the activity log columns 
activity_log_index.columns = ['activity_' + col for col in activity_log_index.columns]
# then replace the first one 
activity_log_index.rename(columns={'activity_userId':'userId'}, inplace = True)

## Curriculum Index
This already has an index but we will make sure that we are able to grab one for every student. The end result of all the indexes is that we should be able to combine all the values for all the indexes into one user data frame which will actually show the output of the data. This data already has a value for all students. Thus we can move on to the next. 


In [22]:
curriculum_indexes  = pd.read_csv(os.path.join(os.getcwd(), '..', 'ENUMA_DATA_CURR', 'curr_indexes.csv'))
curriculum_indexes.columns = ['curr_' + col for col in curriculum_indexes.columns]
curriculum_indexes.rename(columns={'curr_userId':'userId'}, inplace = True)

## Digital Log Index
There are no missing values thus this does not needed to be looked at.

In [23]:
digital_indexes = pd.read_csv(os.path.join(os.getcwd(), '..', 'ENUMA_DATA_DIG', 'user_digital_learning_index.csv'))
digital_indexes.columns = ['dig_' + col for col in digital_indexes.columns]
digital_indexes.rename(columns={'dig_userId':'userId'}, inplace = True)

## Combining all The Data
Now the final step is combiming all the data to with the original users so that we have everything we need for the actual dashboard.
We will do this by left mearning everything with the orriginal users data frame. We will also only be looking at the columns we need for the actual dashboard.
The indexes we will be combining is the 
* activity_learning_index
* dig_DLI
* curr_final_index
Then we need to descretize each of these columns for the coloring of data.

In [24]:
# first drop the columns that will not be useful at all 
enuma_users.drop(columns=['className', 'Loc_X_division_imputed', 'Loc_Y_division_imputed'], inplace=True)
activity_log_index.drop(columns = ['activity_className', 'activity_activeStatus', 'activity_schoolName', 'activity_userRegdate'], inplace=True)
curriculum_indexes.drop(columns = ['curr_schoolName', 'curr_className', 
                                   'curr_activeStatus', 'curr_schoolType', 'curr_schoolArea', 'curr_schoolCluster', 
                                   'curr_userRegdate', 'curr_schoolUID'], inplace = True)
digital_indexes.drop(columns=['dig_schoolName'], inplace = True)

In [25]:
# left join the enuma with the 
final_dashboard_data = enuma_users.merge(
    activity_log_index, how = 'left', on = 'userId'
    ).merge(
        curriculum_indexes, how = 'left', on = 'userId'
        ).merge(digital_indexes, on = 'userId', how='left')

In [26]:
# ok then the next thing we need to do is create a final index and mark all the users with missing values
# we first only keep the active users 
# # then we flag the students that have a missing values for each index 
final_dashboard_data['missing_activity_log_index'] = final_dashboard_data['activity_learning_index'].apply(lambda x:1 if pd.isna(x) else 0)
final_dashboard_data['missing_curr_final_index'] = final_dashboard_data['curr_final_index'].apply(lambda x:1 if pd.isna(x) else 0)
final_dashboard_data['missing_dig_DLI'] = final_dashboard_data['dig_DLI'].apply(lambda x:1 if pd.isna(x) else 0)

# active_users_data = final_dashboard_data.loc[final_dashboard_data['activeStatus'] == True, :]


In [27]:
# Combine index columns into a single final index
def combine_indexes(row):
    indexes = [
        row['activity_learning_index'],
        row['curr_final_index'],
        row['dig_DLI']
    ]
    # Filter out missing values
    valid_indexes = [x for x in indexes if pd.notna(x)]
    
    if valid_indexes:
        return sum(valid_indexes) / len(valid_indexes)
    else:
        return np.nan 

# Apply function to each row
final_dashboard_data['final_combined_index'] = final_dashboard_data.apply(combine_indexes, axis=1)


In [28]:
# we now create the bins for the colors on our data 
def create_index_category(df, colname):
    # Compute percentiles
    p10 = df[colname].quantile(0.10)
    p90 = df[colname].quantile(0.90)
    
    def categorize(x):
        if pd.isna(x):
            return np.nan
        elif x < p10:
            return 'poor'
        elif x > p90:
            return 'gifted'
        else:
            return 'average'
    
    # Create a new column with categories
    category_colname = f'{colname}_category'
    df[category_colname] = df[colname].apply(categorize)

# Apply to each index
for col in ['activity_learning_index', 'curr_final_index', 'dig_DLI', 'final_combined_index']:
    create_index_category(final_dashboard_data, col)


In [29]:
# then we only grab the active students
active_students_dashboard_data = final_dashboard_data.loc[final_dashboard_data['activeStatus'] == True, :]

In [15]:
# push these data frames to the actual local repo
final_dashboard_data.to_csv(os.path.join(os.getcwd(), '..', 'final_aggregated_data','final_aggregated_data.csv'))
active_students_dashboard_data.to_csv(os.path.join(os.getcwd(), '..', 'final_aggregated_data','active_final_aggregated_data.csv'))

## Extracting information for the Main Map
We need four indexes
1. final index
2. activity log index
3. the curriculum index 
4. the dli index 
5. We also need the total number of schools by the division
6. The total number of active students 


We need the medians for each of the indexes, we also need a way to color them. We will color them by just looking at the most frequent count. 

In [30]:
# Count users by Division and final_combined_index_category
counts_for_category_final_index = active_students_dashboard_data.groupby(['Division', 'final_combined_index_category'])['userId'].count()

# Count total users per Division
totals_for_division = active_students_dashboard_data.groupby('Division')['userId'].count()

# Calculate actual percentages 
percentages = counts_for_category_final_index.div(totals_for_division, level=0) * 100
percentages = percentages.round(2).reset_index(name='percentage')

# Pivot to wide format
pivot_df = percentages.pivot(index='Division', columns='final_combined_index_category', values='percentage').fillna(0)

# Composite score logic
def compute_score(row):
    return 2 * row.get('gifted', 0) + 1 * row.get('average', 0) - 2 * row.get('poor', 0)

def assign_color(score):
    if score >= 70:
        return 'green'
    elif score >= 50:
        return 'yellow'
    else:
        return 'red'

pivot_df['composite_score'] = pivot_df.apply(compute_score, axis=1)
pivot_df['color'] = pivot_df['composite_score'].apply(assign_color)


In [31]:
pivot_df

final_combined_index_category,average,gifted,poor,composite_score,color
Division,,,,,
Banbhore,66.67,7.00,6.81,67.05,yellow
Hyderabad,73.91,6.43,6.93,72.91,green
Karachi,50.65,6.12,6.57,49.75,red
Larkana,68.57,4.71,13.14,51.71,yellow
Mirpurkhas,75.03,7.13,5.28,78.73,green
Sehwan,55.35,5.24,7.16,51.51,yellow
Sukkur,74.02,6.32,15.04,56.58,yellow


In [32]:
pivot_df.loc['Banbhore', 'color']

'yellow'

In [33]:
# we need the medians of each of the indexes
main_map_popup_cards = active_students_dashboard_data.groupby(by = 'Division').agg({'schoolName':'nunique',
                                                             'userId':'count',
                                                             'final_combined_index':'median', 
                                                             'activity_learning_index':'median',
                                                             'curr_final_index':'median',
                                                             'dig_DLI':'median'
                                                             })
main_map_popup_cards.columns = ['num_schools', 'num_users', 'final_index', 'activity_index', 'curriculum_index', 'dig']
main_map_popup_cards

,num_schools,num_users,final_index,activity_index,curriculum_index,dig
Division,,,,,,
Banbhore,5,543,0.229836,0.319934,0.063873,0.361492
Hyderabad,20,1805,0.231760,0.320419,0.008691,0.470270
Karachi,44,4415,0.208325,0.322593,-0.054199,0.418861
Larkana,7,700,0.182573,0.327419,-0.035262,0.436782
Mirpurkhas,16,1458,0.242357,0.319695,0.114195,0.353414
Sehwan,16,1411,0.213033,0.337280,-0.106709,0.454077
Sukkur,6,585,0.181838,0.329664,0.009154,0.483109


## Extracting Information For The Subpages 
We need to extract the following. 
* number of operational schools 
* number of closed schools 
* number of missing status schools for TTWF
* total number of historical schools 
* the closed schools  

The data points are messed up right now. We are going to ignore this and fix this tomorrow. The next phase is getting the schoool information down. From now on we are going to do all the processing before we load the dash board since it is starting to take in too much time. For each of the schools we will display. 
* The number of students 
* The dropout rate
* The total index 
* The The activity index 
* The curriculum index 
* The DLI index 
* Then the we will color based off of this

In [34]:
# this is where we will get the division stuff
# we need the operational school based off off this
total_students_and_schools = final_dashboard_data.groupby(by = ['Division']).agg({'schoolName': 'nunique', 'userId':'nunique'}).reset_index()
active_students_and_schools = active_students_dashboard_data.groupby(by = ['Division']).agg({'schoolName':'nunique', 'userId':'nunique'}).reset_index()
division_info_schools_and_students = active_students_and_schools.merge(total_students_and_schools, on = 'Division', how = 'inner')
division_info_schools_and_students['dropoout_students'] = division_info_schools_and_students['userId_y'] - division_info_schools_and_students['userId_x']
division_info_schools_and_students['dropout_rate'] = round(division_info_schools_and_students['dropoout_students'] / division_info_schools_and_students['userId_y'] * 100)
division_info_schools_and_students.drop(columns=['schoolName_y', 'dropoout_students'], inplace = True)
division_info_schools_and_students.columns = ['Division', 'activeSchools', 'activeStudents', 'total_students', 'dropoutRate']
division_info_schools_and_students

,Division,activeSchools,activeStudents,total_students,dropoutRate
0,Banbhore,5,543,877,38.0
1,Hyderabad,20,1805,3846,53.0
2,Karachi,44,4415,11714,62.0
3,Larkana,7,700,896,22.0
4,Mirpurkhas,16,1458,2421,40.0
5,Sehwan,16,1411,3271,57.0
6,Sukkur,6,585,781,25.0


In [35]:
# now we need to fetch all the coordinates for all the schools
division_centers = active_students_dashboard_data.groupby(by = ['Division']).agg(
    {'final_jittered_Loc_X': 'mean', 'final_jittered_Loc_Y':'mean'})

In [36]:
division_centers

,final_jittered_Loc_X,final_jittered_Loc_Y
Division,,
Banbhore,24.899141,67.389295
Hyderabad,25.232237,68.328089
Karachi,24.688897,67.205096
Larkana,24.916860,67.375133
Mirpurkhas,24.904102,67.382326
Sehwan,26.249958,68.407205
Sukkur,24.955669,67.345348


In [37]:
# we will use the active students, the total students and the dropout_rate
# and we will use the 
# we need to save this in the same directory as the pages 
total_students_and_schools = final_dashboard_data.groupby(by = ['Division', 'schoolName']).agg({'userId':'nunique'}).reset_index()
active_students_and_schools = active_students_dashboard_data.groupby(by = ['Division', 'schoolName']).agg({'userId':'nunique'}).reset_index()
division_info_schools_and_students = active_students_and_schools.merge(total_students_and_schools, on = ['Division', 'schoolName'], how = 'inner')
division_info_schools_and_students.columns = ['Division', 'schoolName', 'num_active_students', 'num_total_students']
division_info_schools_and_students['dropout_students'] = division_info_schools_and_students['num_total_students'] - division_info_schools_and_students['num_active_students']
division_info_schools_and_students['dropout_rate'] = round(
    division_info_schools_and_students['dropout_students'] / division_info_schools_and_students['num_total_students'] * 100
    )
division_info_schools_and_students.drop(columns=['dropout_students'], inplace = True)
division_info_schools_and_students.to_csv(os.path.join(os.getcwd(), '..', 'school_popup_card_info', 'division_info_schools_and_students.csv'))

In [38]:
division_info_schools_and_students.loc[division_info_schools_and_students['Division'] == 'Banbhore', :]

,Division,schoolName,num_active_students,num_total_students,dropout_rate
0,Banbhore,216-MS-BNB-THT,109,162,33.0
1,Banbhore,217-MS-BNB-THT,102,179,43.0
2,Banbhore,218-MS-BNB-THT,100,192,48.0
3,Banbhore,219-MS-BNB-THT,106,163,35.0
4,Banbhore,220-MS-BNB-THT,126,181,30.0


In [39]:
def colorize(category):
    if category == 'average':
        return 'orange'
    elif category == 'gifted':
        return 'green'
    else:
        return 'red'

In [40]:
# the next thing we need to get are the indexes by the schools 
# we will only look at the active ones 
def safe_mode(series):
    mode = series.mode(dropna=True)
    if not mode.empty:
        return mode.iloc[0]
    else:
        return np.nan  # or 'Unknown', 'Missing', etc.

grouped = active_students_dashboard_data.groupby(by=['Division', 'schoolName']).agg({
    'final_combined_index': 'median',
    'activity_learning_index': 'median',
    'curr_final_index': 'median',
    'dig_DLI': 'median',
    'activity_learning_index_category': safe_mode,
    'curr_final_index_category': safe_mode,
    'dig_DLI_category': safe_mode,
    'final_combined_index_category':safe_mode
})

# apply colors 
# Apply colorize to each category column and create new color columns

# lets create colors for the grouped one so that we can plot this on the 

grouped['final_combined_index_color'] = grouped['final_combined_index_category'].apply(colorize)

In [41]:
def colorize_category(category):
    if (category == 'average'):
        return 'orange'
    elif (category == 'gifted'):
        return 'green'
    else:
        return 'red'

In [42]:
columns_to_colorize = ['activity_learning_index_category', 'curr_final_index_category','dig_DLI_category']

In [43]:
grouped['curr_final_index_color'] = grouped['curr_final_index_category'].apply(colorize_category)
grouped['dig_DLI_category_color'] = grouped['dig_DLI_category'].apply(colorize_category)
grouped['activity_learning_index_category_color'] = grouped['activity_learning_index_category'].apply(colorize_category)

In [30]:
grouped.columns

Index(['final_combined_index', 'activity_learning_index', 'curr_final_index',
       'dig_DLI', 'activity_learning_index_category',
       'curr_final_index_category', 'dig_DLI_category',
       'final_combined_index_category', 'final_combined_index_color',
       'curr_final_index_color', 'dig_DLI_category_color',
       'activity_learning_index_category_color'],
      dtype='object')

In [44]:
# now we read this into the subpages 
grouped.to_csv(os.path.join(os.getcwd(), '..', 'school_popup_card_info', 'school_popup_card_info.csv'))

In [45]:
grouped.loc[('Karachi'), :]

,final_combined_index,activity_learning_index,curr_final_index,dig_DLI,activity_learning_index_category,curr_final_index_category,dig_DLI_category,final_combined_index_category,final_combined_index_color,curr_final_index_color,dig_DLI_category_color,activity_learning_index_category_color
schoolName,,,,,,,,,,,,
101-MS-KHI-MLR,0.208484,0.317772,0.026100,0.510870,average,average,average,average,orange,orange,orange,orange
103-MS-KHI-MLR,0.181099,0.310380,-0.101164,0.431426,average,average,average,average,orange,orange,orange,orange
104-MS-KHI-MLR,0.181628,0.300699,-0.107135,0.460033,average,average,average,average,orange,orange,orange,orange
108-MS-KHI-MLR,0.133145,0.295247,-0.219219,0.306136,average,average,average,average,orange,orange,orange,orange
109-MS-KHI-MLR,0.209699,0.307495,-0.033815,0.422619,average,average,average,average,orange,orange,orange,orange
110-MS-KHI-MLR,0.198327,0.323423,-0.108936,0.446272,average,average,average,average,orange,orange,orange,orange
111-MS-KHI-MLR,0.149649,0.323077,-0.159277,0.463303,average,average,average,average,orange,orange,orange,orange
112-MS-KHI-MLR,0.171703,0.316218,-0.139273,0.486188,average,average,average,average,orange,orange,orange,orange
114-MS-KHI-MLR,0.317531,0.326250,0.166124,0.456790,average,average,average,average,orange,orange,orange,orange


## Extracting User Information 
The next and the final thing we have to do is extract the school level information! Ok here is something that we can do. We need do not need the aggregate information. Instead if we can think of any pie charts or anything like that then we will incorporate this. 

In [46]:
columns_to_categorize = [
    'activity_completion_rate', 'activity_avg_session_gap', 'activity_max_session_gap',
    'activity_last_session_gap', 'activity_num_gaps_gt_3', 'activity_num_gaps_gt_7',
    'activity_lessons_attempted_per_day', 'activity_lessons_completed_per_day',
    'activity_consistency_score', 'activity_learning_index', 'curr_placement_index',
    'curr_placement_score_eng', 'curr_placement_score_mat', 'curr_placement_level_eng',
    'curr_placement_level_mat', 'curr_num_placement_tests', 'curr_learning_index_eng',
    'curr_learning_index_mat', 'curr_raw_learning_rate_eng', 'curr_raw_learning_rate_mat',
    'curr_engagement_index', 'curr_num_sessions', 'curr_raw_engagement_score', 'dig_completion_weight'
]

def categorize_by_percentiles(df, columns):
    """
    For each column in 'columns', computes the 10th and 90th percentiles and creates a new
    column labeling each value as 'gifted', 'average', or 'needs attention'.
    """
    for col in columns:
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in DataFrame. Skipping.")
            continue
        
        # Calculate thresholds
        q10 = df[col].quantile(0.10)
        q90 = df[col].quantile(0.90)

        # Create category column
        def label_value(x):
            if pd.isnull(x):
                return None
            elif x < q10:
                return 'needs attention'
            elif x > q90:
                return 'gifted'
            else:
                return 'average'
        
        df[f'{col}_category'] = df[col].apply(label_value)
    
    return df
final_dashboard_data = categorize_by_percentiles(final_dashboard_data, columns_to_categorize)

In [47]:
active_students_dashboard_data = final_dashboard_data.loc[final_dashboard_data['activeStatus'] == True, :]

In [48]:
active_students_dashboard_data.columns

Index(['userId', 'schoolName', 'activeStatus', 'userRegdate', 'schoolUID',
       'schoolType', 'schoolArea', 'schoolCluster', 'Division', 'District',
       'missing_division', 'final_jittered_Loc_X', 'final_jittered_Loc_Y',
       'activity_num_lessons_attempted', 'activity_num_lessons_completed',
       'activity_active_days', 'activity_completion_rate',
       'activity_eng_max_course', 'activity_eng_max_level',
       'activity_eng_max_unit', 'activity_eng_max_lesson',
       'activity_mat_max_course', 'activity_mat_max_level',
       'activity_mat_max_unit', 'activity_mat_max_lesson',
       'activity_eng_course_final', 'activity_eng_level_final',
       'activity_eng_unit_final', 'activity_eng_lesson_final',
       'activity_mat_course_final', 'activity_mat_level_final',
       'activity_mat_unit_final', 'activity_mat_lesson_final',
       'activity_avg_session_gap', 'activity_max_session_gap',
       'activity_last_session_gap', 'activity_num_gaps_gt_3',
       'activity_num_ga

In [49]:
# now we add the color 
def add_color_columns(df, columns):
    """
    For each *_category column corresponding to the input columns, 
    adds a *_color column with:
        - 'green' for 'gifted'
        - 'orange' for 'average'
        - 'red' for 'needs attention'
    """
    category_to_color = {
        'gifted': 'green',
        'average': 'orange',
        'needs attention': 'red'
    }

    for col in columns:
        category_col = f'{col}_category'
        color_col = f'{col}_color'
        if category_col in df.columns:
            df[color_col] = df[category_col].apply(
                lambda x: category_to_color.get(x, '') if pd.notnull(x) else ''
            )
        else:
            print(f"Warning: Category column '{category_col}' not found in DataFrame.")
    
    return df
active_students_dashboard_data = active_students_dashboard_data = add_color_columns(
    active_students_dashboard_data, columns_to_categorize)

C:\Users\azpee\AppData\Local\Temp\ipykernel_23640\592541992.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[color_col] = df[category_col].apply(
C:\Users\azpee\AppData\Local\Temp\ipykernel_23640\592541992.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[color_col] = df[category_col].apply(
C:\Users\azpee\AppData\Local\Temp\ipykernel_23640\592541992.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [51]:
# now we need to figure out how to make a pie chart 
active_students_dashboard_data.head()

,userId,schoolName,activeStatus,userRegdate,schoolUID,schoolType,schoolArea,schoolCluster,Division,District,...,curr_placement_level_mat_color,curr_num_placement_tests_color,curr_learning_index_eng_color,curr_learning_index_mat_color,curr_raw_learning_rate_eng_color,curr_raw_learning_rate_mat_color,curr_engagement_index_color,curr_num_sessions_color,curr_raw_engagement_score_color,dig_completion_weight_color
0,287925c0-b603-11ef-a0d0-2df3ae335ef0,011-MS-LHR-CNT,True,2024-12-09 07:56:55,11,MS,LHR,CNT,Sehwan,NaN,...,,,,,,,,,,
1,463374b0-a7c4-11ee-af18-718b1892f16d,108-MS-KHI-MLR,True,2023-12-31 10:06:32,108,MS,KHI,MLR,Karachi,Malir,...,orange,orange,orange,orange,orange,orange,orange,green,orange,
2,49098cd0-06b3-11ef-9919-694f18076f12,016-MS-LHR-CNT,True,2024-04-30 05:34:16,16,MS,LHR,CNT,Sehwan,NaN,...,orange,orange,orange,orange,orange,orange,orange,orange,orange,orange
3,151dbc20-37a0-11ef-9f89-896b67520125,162-MS-HYD-HYD,True,2024-07-01 11:50:16,162,MS,HYD,HYD,Hyderabad,Hyderabad,...,orange,orange,orange,orange,orange,orange,red,orange,red,
4,00b031d0-a011-11ef-97e0-4510b1bb181f,184-MS-KHI-MLR,True,2024-11-11 09:40:35,184,MS,KHI,MLR,Karachi,Malir,...,orange,orange,orange,orange,orange,orange,green,green,green,


In [69]:
# first get the aggregated data for the total active students in a school 
# WE NEED TO SAVE THIS CSV AS A FILE
active_students_per_school = active_students_dashboard_data.groupby('schoolName').agg({'userId': 'nunique'}).reset_index()
active_students_per_school.to_csv(os.path.join('..', 'school_aggregated_data', 'active_students_per_school.csv'))

In [61]:
# then the next this we need is the aggregation for the pie chart visuals 
pie_chart_columns = ['activity_learning_index_category','curr_final_index_category,dig_DLI_category','final_combined_index_category']


active_students_dashboard_data_ali_cat = active_students_dashboard_data.groupby('schoolName')['activity_learning_index_category'] \
    .value_counts(normalize=True) \
    .unstack(fill_value=0)


active_students_dashboard_data_cfi_cat = active_students_dashboard_data.groupby('schoolName')['curr_final_index_category'] \
    .value_counts(normalize=True) \
    .unstack(fill_value=0)

active_students_dashboard_data_dig_cat = active_students_dashboard_data.groupby('schoolName')['dig_DLI_category'] \
    .value_counts(normalize=True) \
    .unstack(fill_value=0)


active_students_dashboard_data_fin_cat = active_students_dashboard_data.groupby('schoolName')['final_combined_index_category'] \
    .value_counts(normalize=True) \
    .unstack(fill_value=0)

In [62]:
distribution_pct = (
    active_students_dashboard_data_ali_cat.add_suffix('_activity')
    .join(active_students_dashboard_data_cfi_cat.add_suffix('_curriculum'))
    .join(active_students_dashboard_data_dig_cat.add_suffix('_dig'))
    .join(active_students_dashboard_data_fin_cat.add_suffix('_fin'))
)


In [70]:
distribution_pct = distribution_pct.reset_index()
distribution_pct.to_csv(os.path.join('..', 'school_aggregated_data', 'distribution_pct.csv'))

In [68]:
# now we will go how to make an example pie chart 
import plotly.express as px
activity_log_columns = ['average_activity', 'gifted_activity', 'needs attention_activity'] 
curriculum_log_columns = ['average_curriculum', 'gifted_curriculum', 'poor_curriculum']
dig_log_columns = ['average_dig', 'gifted_dig', 'poor_dig']
fin_index_columns = ['average_fin', 'gifted_fin', 'poor_fin']
school_row = distribution_pct.loc[distribution_pct['schoolName'] == '003-MS-KHI-MLR', activity_log_columns].squeeze()
fig = px.pie(
    names=school_row.index.str.replace('_activity', ''),  # Clean label if needed
    values=school_row.values,
    title='Activity Learning Index Distribution'
)
fig.show()

In [75]:
columns_to_display_activity = ['userId', 'userRegdate', 
                      'activity_completion_rate', 
                      'activity_eng_course_final',
                      'activity_eng_level_final',
                      'activity_eng_unit_final',
                      'activity_eng_lesson_final',
                      'activity_mat_course_final',
                      'activity_mat_level_final',
                      'activity_mat_unit_final',
                      'activity_mat_lesson_final',
                      'activity_last_session_gap',
                      'activity_eng_perf_bin',
                      'activity_mat_perf_bin',
                      'activity_is_mature',
                      'activity_consistency_score',
                      'missing_activity_log_index',
                      'activity_learning_index_category',
                      'activity_completion_rate_category',
                      'activity_avg_session_gap_category',
                      'activity_max_session_gap_category',
                      'activity_last_session_gap_category',
                      'activity_num_gaps_gt_3_category',
                      'activity_num_gaps_gt_7_category',
                      'activity_lessons_attempted_per_day_category',
                      'activity_lessons_completed_per_day_category',
                      'activity_consistency_score_category',
                      'activity_completion_rate_color',
                      'activity_avg_session_gap_color',
                      'activity_max_session_gap_color',
                      'activity_last_session_gap_color',
                      'activity_num_gaps_gt_3_color',
                      'activity_num_gaps_gt_7_color',
                      'activity_lessons_attempted_per_day_color',
                      'activity_lessons_completed_per_day_color',
                      'activity_consistency_score_color',
                      'activity_learning_index_color'







                      ]

columns_to_display_curr = [
    'curr_placement_index',
    'curr_placement_score_eng',
    'curr_placement_score_mat',
    'curr_placement_level_eng',
    'curr_placement_level_mat',
    'curr_learning_index_overall',
    'curr_learning_index_eng',
    'curr_learning_index_mat',
    'curr_raw_learning_rate_eng',
    'curr_raw_learning_rate_mat',
    'curr_engagement_index',
    'curr_num_sessions',
    'curr_num_problems_attempted',
    'curr_active_days',
    'curr_days_between',
    'curr_raw_engagement_score',
    'curr_weight_sum',
    'curr_final_index',
    'curr_student_group',
    'missing_curr_final_index',
    'curr_final_index_category',
    'curr_placement_index_category',
    'curr_placement_score_eng_category',
    'curr_placement_score_mat_category',
    'curr_placement_level_eng_category',
    'curr_placement_level_mat_category',
    'curr_num_placement_tests_category',
    'curr_learning_index_eng_category',
    'curr_learning_index_mat_category',
    'curr_raw_learning_rate_eng_category',
    'curr_raw_learning_rate_mat_category',
    'curr_engagement_index_category',
    'curr_num_sessions_category',
    'curr_raw_engagement_score_category',
    'curr_placement_index_color',
    'curr_placement_score_eng_color',
    'curr_placement_score_mat_color',
    'curr_placement_level_eng_color',
    'curr_placement_level_mat_color',
    'curr_num_placement_tests_color',
    'curr_learning_index_eng_color',
    'curr_learning_index_mat_color',
    'curr_raw_learning_rate_eng_color',
    'curr_raw_learning_rate_mat_color',
    'curr_engagement_index_color',
    'curr_num_sessions_color',
    'curr_raw_engagement_score_color']

columns_to_display_dig = ['dig_total_attempted', 'dig_total_correct','dig_DLI','missing_dig_DLI','dig_DLI_category']

columns_to_diplay_final = ['final_combined_index', 'final_combined_index_category']

userId
schoolName
activeStatus
userRegdate
schoolUID
schoolType
schoolArea
schoolCluster
Division
District
missing_division
final_jittered_Loc_X
final_jittered_Loc_Y
activity_num_lessons_attempted
activity_num_lessons_completed
activity_active_days
activity_completion_rate
activity_eng_max_course
activity_eng_max_level
activity_eng_max_unit
activity_eng_max_lesson
activity_mat_max_course
activity_mat_max_level
activity_mat_max_unit
activity_mat_max_lesson
activity_eng_course_final
activity_eng_level_final
activity_eng_unit_final
activity_eng_lesson_final
activity_mat_course_final
activity_mat_level_final
activity_mat_unit_final
activity_mat_lesson_final
activity_avg_session_gap
activity_max_session_gap
activity_last_session_gap
activity_num_gaps_gt_3
activity_num_gaps_gt_7
activity_eng_perf_bin
activity_mat_perf_bin
activity_is_mature
activity_lessons_attempted_per_day
activity_lessons_completed_per_day
activity_consistency_score
activity_learning_index
curr_placement_index
curr_placem

In [78]:
active_students_dashboard_data.to_csv(os.path.join(os.getcwd(), '..', 'final_aggregated_data', 'student_data_total.csv'))